In [345]:
import imdb
import pandas as pd 
import numpy as np
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

In [109]:
class Timer(object):  
    def start(self):  
        if hasattr(self, 'interval'):  
            del self.interval  
        self.start_time = time.time()  
  
    def stop(self):  
        if hasattr(self, 'start_time'):  
            self.interval = time.time() - self.start_time  
            del self.start_time 

In [301]:
# Create the object that will be used to access the IMDb's database.
ia = imdb.IMDb() # by default access the web.

# Search for a movie (get a list of Movie objects).
s_result = ia.search_movie('Up Close and Personal (1996)')
#s_result
myID=s_result[0].movieID
item=ia.get_movie(myID)

In [344]:
#print 'genre==>',item['genre']
#print 'year==>',item['year']
#print 'kind==>',item['kind']
#print 'director==>',item['director']
#print 'cast==>',item['cast']
#print 'writer==>',item['writer']
#print 'rating==>',item['rating']
#print 'runtimes==>',item['runtimes']
#print 'countries==>',item['countries']
#print 'languages==>',item['languages']
#print 'production companies==>',item['production companies']


In [347]:
#test sur notre base:
movies = pd.read_csv('./ml-latest-small/movies.csv')
newMovies=movies[['movieId','title']]
newMovies['genre']=np.NaN
newMovies['year']=np.NaN
newMovies['writer']=np.NaN
newMovies['director']=np.NaN
newMovies['cast']=np.NaN
newMovies['rating']=np.NaN
newMovies['runtimes']=np.NaN
newMovies['countries']=np.NaN
newMovies['languages']=np.NaN
newMovies['production companies']=np.NaN

In [400]:
time0 = Timer()
time0.start()
#for index, row in newMovies.ix[:200,:].iterrows():
for index, row in newMovies.ix[2000:3000,:].iterrows():
    title=row['title']
    time1 = Timer()
    time1.start()
    ###
    s_result=ia.search_movie(title)
    ###
    time1.stop()
    print 'ia.search_movie: ', time1.interval
    if (s_result):
        theID=s_result[0].movieID
        time2 = Timer()
        time2.start()
        ###
        item=ia.get_movie(theID)
        ###
        time2.stop()
        print 'ia.get_movie: ', time2.interval
        
        if( 'genres'  in item.keys()):
            newMovies.ix[index,'genre']=','.join(item['genre'])
        if( 'year'  in item.keys()):
            newMovies.ix[index,'year']=item['year']
        if( 'rating'  in item.keys()):
            newMovies.ix[index,'rating']=item['rating']
        if( 'runtimes'  in item.keys()):
            newMovies.ix[index,'runtimes']=item['runtimes'][0]
        ###
        if( 'countries'  in item.keys()):
            countries=''
            for i in item['countries']:
                countries += i +','
            newMovies.ix[index,'countries']=countries
        ###
        if( 'languages'  in item.keys()):
            languages=''
            for i in item['languages']:
                languages +=i +','
            newMovies.ix[index,'languages']=languages
        ###
        if( 'writer'  in item.keys()):
            Lwriter=''
            for i in item['writer']:
                Lwriter +=i['name']+','
            newMovies.ix[index,'writer']=Lwriter
        ###
        if( 'director'  in item.keys()):
            Ldirector=''
            for i in item['director']:
                Ldirector +=i['name']+','
            newMovies.ix[index,'director']=Ldirector
        ###
        if( 'cast'  in item.keys()):
            Lcast=''
            for i in item['cast']:
                Lcast +=i['name']+','
            newMovies.ix[index,'cast']=Lcast
        ###
        if( 'production companies'  in item.keys()):
            Lproduction_company=''
            for i in item['production companies']:
                Lproduction_company +=i['name']+','
            newMovies.ix[index,'production companies']=Lproduction_company
    else:
        pass
    
    
    
time0.stop()
print 'whole iter: ', time0.interval

ia.search_movie:  0.746999979019
ia.get_movie:  2.70600008965
ia.search_movie:  0.460999965668
ia.get_movie:  1.74899983406
ia.search_movie:  0.400000095367
ia.get_movie:  10.5360000134
ia.search_movie:  0.605999946594
ia.get_movie:  2.55800008774
ia.search_movie:  0.665000200272
ia.get_movie:  2.16899991035
ia.search_movie:  1.06599998474
ia.get_movie:  2.72099995613
ia.search_movie:  0.711999893188
ia.get_movie:  2.21100020409
ia.search_movie:  0.817000150681
ia.get_movie:  1.62800002098
ia.search_movie:  1.01400017738
ia.get_movie:  2.21199989319
ia.search_movie:  0.700000047684
ia.get_movie:  1.64699983597
ia.search_movie:  0.694000005722
ia.get_movie:  2.1930000782
ia.search_movie:  0.637000083923
ia.get_movie:  3.63899993896
ia.search_movie:  0.500999927521
ia.get_movie:  2.71300005913
ia.search_movie:  0.450999975204
ia.get_movie:  1.96499991417
ia.search_movie:  0.705000162125
ia.get_movie:  2.39999985695
ia.search_movie:  0.389000177383
ia.get_movie:  2.3069999218
ia.search_mo

In [401]:
#newMovies[(newMovies.year.isnull()) & (~ newMovies.genre.isnull())]
#newMovies.ix[:200,:].isnull().sum()
#newMovies[(newMovies.genre.isnull())]
MoviesFeature=newMovies.ix[0:3000,:]
MoviesFeature[(MoviesFeature.genre.isnull())]
#pour ces films on a pas pu les trouver a cause du title: il est peut être mal ecrit  et donc ia.search_movie ne les trouve pas 

,movieId,title,genre,year,writer,director,cast,rating,runtimes,countries,languages,production companies
28,29,"City of Lost Children, The (Cité des enfants p...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,47,Seven (a.k.a. Se7en) (1995),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,73,"Misérables, Les (1995)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,108,Catwalk (1996),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,112,Rumble in the Bronx (Hont faan kui) (1995),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,124,"Star Maker, The (Uomo delle stelle, L') (1995)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187,213,Burnt by the Sun (Utomlyonnye solntsem) (1994),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,264,"Enfer, L' (1994)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [402]:
MoviesFeature.to_csv('./ml-latest-small/MoviesFeatures.csv',encoding='utf_8')#exporter le dataFrame en un fichier CSV